# Exploratory Data Analysis for Covid Prediction

Disclaimer: Results and methods might change in the future. Unexpectedly for me, I might even change the topic if I find that I don't have enough tools for "several models" requirement or if I find that I need full dataset not provided by competition I am using as a base.

## Package imports

In [160]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px

## Importing data

### Data description

Kaggle competition provides 3 files, but only 1 will be used here:

**train.csv** - contains unique id of each datapoint, name of county and province/state (if applicable) where this datapoint was taken, date, number of confirmed cases, and number of fatalities - both seem to be cumulative.

In [230]:
# Read the data
DATA_DIR = "./data"

df = pd.read_csv(f"{DATA_DIR}/train.csv").set_index("Id")

# Format date and time
df["Date"] = pd.to_datetime(df["Date"])

### First look at the data

In [231]:
print(f"Shape: {df.shape}")
df.head()

Shape: (35995, 5)


,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
Id,,,,,
1,NaN,Afghanistan,2020-01-22,0.0,0.0
2,NaN,Afghanistan,2020-01-23,0.0,0.0
3,NaN,Afghanistan,2020-01-24,0.0,0.0
4,NaN,Afghanistan,2020-01-25,0.0,0.0
5,NaN,Afghanistan,2020-01-26,0.0,0.0


In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35995 entries, 0 to 35994
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Id              35995 non-null  int64         
 1   Province_State  15295 non-null  object        
 2   Country_Region  35995 non-null  object        
 3   Date            35995 non-null  datetime64[ns]
 4   ConfirmedCases  35995 non-null  float64       
 5   Fatalities      35995 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 1.6+ MB


### How many countries are there?

In [19]:
print(f'{df["Country_Region"].nunique()} unique countries/regions')

184 unique countries/regions


### Duration of the data

In [22]:
df["Date"].describe()

count                            35995
mean     2020-03-19 00:00:00.000000256
min                2020-01-22 00:00:00
25%                2020-02-19 00:00:00
50%                2020-03-19 00:00:00
75%                2020-04-17 00:00:00
max                2020-05-15 00:00:00
Name: Date, dtype: object

As we can see, data is distributed between Jan 22, 2020 and May 15, 2020, taken once per day

### Any missing data?

In [24]:
# Null values in absolute numbers
df.isna().sum() 

Id                    0
Province_State    20700
Country_Region        0
Date                  0
ConfirmedCases        0
Fatalities            0
dtype: int64

In [25]:
# Null values in percent
df.isna().sum() * 100 / len(df)

Id                 0.000000
Province_State    57.507987
Country_Region     0.000000
Date               0.000000
ConfirmedCases     0.000000
Fatalities         0.000000
dtype: float64

As we can see, there is no missing data from most columns, but 57.5% province/state column is empty due to many countries not having provinces or not publishing individual nubers for them.

Due to that, I I might not be using it for predictions, leaving it at country-level precision.

### Duplicates

In [30]:
# Full duplicates
df.duplicated(subset=["Province_State", "Country_Region", "Date", "ConfirmedCases", "Fatalities"]).sum()

0

In [31]:
# Up to province
df.duplicated(subset=["Country_Region", "Date", "ConfirmedCases", "Fatalities"]).sum()

4775

The number of duplicates depends on your precision. There are no full duplicates, indicating that the data is already cleaned and prepared, and 4,775 entries where only province is different (e.g. entries "Connecticut / US / 22.01.2020 / 0 / 0" and "Colorado / US / 22.01.2020/ 0 / 0" are considered duplicates here).

### Data cleaning and preparation

All merging and duplicate removal were done by the data source (Kaggle). However, to ensure easier experience and remove empty values, we can gp two ways:

1. Reduce number of data by combining all provinces into country and summing their case counts.
2. Combine "Country_Region" and "Province_State" into one column with "Country: State" format.

We will go with the second option as it provides more datapoints, but also make the first since it also has some interesting data.

In [286]:
# Option 1

df_countries = df.drop("Province_State", axis=1)
df_countries = df_countries.groupby(["Country_Region", "Date"]).sum()
df_countries = df_countries.reset_index()

In [165]:
#TODO try to make it look better
# Option 2

df["States"] = ''
for index, row in df.iterrows():
    df.at[index, "States"] = f"{row['Country_Region']}" + f"{': ' + row['Province_State'] if pd.isna(row['Province_State']) == False else ''}"

Since I am planning on using regression and it doesn't play well with the dates, we will need another step - to transfrom dates into "days since Jan 22" format

In [353]:
df["Days"] = (df['Date'] - df['Date'].min())  / np.timedelta64(1,'D')

df_countries["Days"] = df_countries["Date"] - pd.to_datetime("2020-01-22")

## Plots

### Worldwide cases and fatalities
Let's plot number of COVID cases and fatalities worldwide to get the large picture

In [315]:
# Sum data for the world
df_world = df[["Date", "ConfirmedCases", "Fatalities"]].groupby("Date").sum()

px.line(df_world)

Now let's see the graphs for all countries

In [288]:
px.line(df_countries, x = "Date", y = "ConfirmedCases", color = "Country_Region")

What's interesting is that after March US has had massive lead in cases over other countries and that number of cases actually seems to be distributed in such a way that linear regression might roughly work. Remembering how it developed later, I came here expecting a time series instead of a steadily rising graph, but I guess it was just the very beginning, so no cycles formed yet.

We can also take a look at the data without US' outlier:

In [ ]:
px.line(df_countries, x = "Date", y = "ConfirmedCases", color = "Country_Region")

As we can see, pandemic really did start in whole world. Interesting outlier here is China, which entered plateau around middle of February at around 72k confirmed (and reported) cases.

## Modeling

Disclaimer: This section will see heavy updates over time as I add more models. For now, I will only build models for the world and number of cases, and expand them to individual countries later on.

Now for the more practical part of the project: the actual models and predictions. First of all, we need to split the datasets. Unfortunately, we can't use random split, since this is a time series. Therefore, we must find a cutoff point - April 30 - and try to use points in Jan 22 - Apr 30 for training and May 1 - May 15 for testing. Final evaluation can be done with either using expanded dataset or by submitting the results on Kaggle.

In [294]:
from sklearn.linear_model import LinearRegression

In [355]:
df_world = df[["Days", "ConfirmedCases", "Fatalities"]].groupby("Days").sum().reset_index()

,Days,ConfirmedCases,Fatalities
0,0.0,554.0,17.0
1,1.0,653.0,18.0
2,2.0,939.0,26.0
3,3.0,1432.0,42.0
4,4.0,2113.0,56.0


In [359]:
df_train = df_world.loc[df_world['Days'] <= 99]
X_train = df_train[["Days"]]
y_train_cases = df_train["ConfirmedCases"]
y_train_deaths = df_train["Fatalities"]

df_test = df_world.loc[df_world['Days'] > 99]
X_test = df_test[["Days"]]
y_test_cases = df_test["ConfirmedCases"]
y_test_deaths = df_test["Fatalities"]

In [360]:
linreg = LinearRegression()
linreg.fit(X_train, y_train_cases)
pred = linreg.predict(X_test)

In [401]:
predictions = df_world[["Days", "ConfirmedCases"]]

In [412]:
pred_bis = pd.DataFrame(pred)
pred_bis["Index"] = np.arange(100, 115)
pred_bis = pred_bis.set_index("Index")

In [413]:
pred_bis

,0
Index,
100,2.218654e+06
101,2.248007e+06
102,2.277359e+06
103,2.306711e+06
104,2.336063e+06
105,2.365415e+06
106,2.394767e+06
107,2.424119e+06
108,2.453471e+06


In [414]:
predictions["preds"] = pred_bis

In [415]:
predictions

,Days,ConfirmedCases,preds
0,0.0,554.0,NaN
1,1.0,653.0,NaN
2,2.0,939.0,NaN
3,3.0,1432.0,NaN
4,4.0,2113.0,NaN
...,...,...,...
110,110.0,4176589.0,2.512175e+06
111,111.0,4260955.0,2.541527e+06
112,112.0,4345950.0,2.570879e+06
113,113.0,4440974.0,2.600231e+06


In [419]:
px.line(predictions, x="Days", y=["ConfirmedCases", "preds"])

Evidently, linear regression doesn't work since it has error of about 1.5M cases. I will have to use ARIMA then, but it will be added sometime later